# Load wandb metrics and create nicer plots

In [11]:
%load_ext autoreload
%autoreload 2

In [35]:
import os
from copy import deepcopy
from pathlib import Path

import dotenv
import molfeat
import pandas as pd
import torch
import torch.nn as nn
from hydra import compose, initialize
from hydra.core.global_hydra import GlobalHydra
from hydra.utils import instantiate
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import WandbLogger
from omegaconf import DictConfig, OmegaConf, open_dict

import wandb
from src import utils
from src.models.jump_cl import BasicJUMPModule
from src.utils import instantiate_evaluator_list

In [13]:
for i in range(1, 4):
    if not Path(f"../cpjump{i}/jump/").exists():
        print(f"Mounting cpjump{i}...")
        os.system(f"sshfs bioclust:/projects/cpjump{i}/ ../cpjump{i}")
    else:
        print(f"cpjump{i} already mounted.")

Mounting cpjump1...
Mounting cpjump2...
Mounting cpjump3...


In [14]:
run = "2023-08-17_13-32-50/0"
epoch = 41

In [15]:
ckpt_str = "../cpjump1/jump/logs/train/multiruns/{run}/checkpoints/epoch_{epoch:0>3}.ckpt"

run_dict = {
    "small1": (run := "2023-08-16_11-59-26/0", "small_jump_cl", epoch := 43, ckpt_str.format(run=run, epoch=epoch)),
    "small": (run := "2023-08-17_13-32-50/0", "small_jump_cl", epoch := 41, ckpt_str.format(run=run, epoch=epoch)),
    "med": (run := "2023-08-07_11-55-54", "med_jump_cl", epoch := 5, ckpt_str.format(run=run, epoch=epoch)),
    "big": (run := "2023-08-01_11-37-40", "big_jump_cl", epoch := 1, ckpt_str.format(run=run, epoch=epoch)),
}

In [16]:
run, experiment, epoch, ckpt = run_dict["small"]

In [17]:
os.system(f"cat ../cpjump1/jump/logs/train/multiruns/{run}/.hydra/overrides.yaml");

- experiment=small_jump_cl
- trainer.devices=[0,2]
- data.num_workers=16
- data.prefetch_factor=2
- model/criterion=frozen_contrastive.yaml


In [18]:
os.listdir(f"../cpjump1/jump/logs/train/multiruns/{run}/checkpoints")

['last.ckpt', 'epoch_041.ckpt']

In [19]:
initialize(version_base=None, config_path="../configs/")

hydra.initialize()

In [20]:
run, experiment, epoch, ckpt = run_dict["small"]

In [21]:
cfg = compose(
    config_name="train.yaml",
    overrides=[
        "evaluate=true",
        "eval=retrieval",
        "paths.projects_dir=..",
        f"paths.output_dir=../cpjump1/jump/logs/train/multiruns/{run}",
        "experiment=fp_small",
        "data.batch_size=4",
        # "model/molecule_encoder=gin_masking.yaml",
        "trainer.devices=1",
        # "eval.moa_image_task.datamodule.data_root_dir=../",
    ],
)
print(OmegaConf.to_yaml(cfg))

task_name: train
tags:
- small_jump_cl
- fingerprints
- clip_like
- ${model.image_encoder.instance_model_name}
train: true
test: true
evaluate: true
compile: false
ckpt_path: null
seed: 12345
data:
  compound_transform:
    _target_: src.modules.compound_transforms.fp_transform.FPTransform
    fps:
    - maccs
    - ecfp
    compound_str_type: inchi
    params:
      ecfp:
        radius: 2
  _target_: src.models.jump_cl.datamodule.BasicJUMPDataModule
  batch_size: 4
  num_workers: 24
  pin_memory: null
  prefetch_factor: 3
  drop_last: true
  transform:
    _target_: src.modules.transforms.DefaultJUMPTransform
    _convert_: object
    size: 128
    dim:
    - -2
    - -1
  force_split: false
  splitter:
    _target_: src.splitters.ScaffoldSplitter
    train: 800
    test: 200
    val: 100
    retrieval: 0
  use_compond_cache: false
  data_root_dir: ${paths.projects_dir}/
  split_path: ${paths.split_path}/fp_small/
  dataloader_config:
    train:
      batch_size: ${data.batch_size}
 

In [61]:
dm = instantiate(cfg.data)

/home/gwatk/miniconda3/envs/jump_models/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/gwatk/miniconda3/envs/jump_models/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Plea

In [63]:
dm.prepare_data()

In [64]:
dm.setup("train")

In [65]:
dm.val_dataset

MoleculeImageDataset(n_compounds=100, n_images=2860)

In [67]:
dl = dm.val_dataloader()

/home/gwatk/miniconda3/envs/jump_models/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [68]:
b = next(iter(dl))

/home/gwatk/miniconda3/envs/jump_models/lib/python3.10/site-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 24 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [74]:
cfg.model["_target_"] += ".load_from_checkpoint"
with open_dict(cfg.model):
    cfg.model["checkpoint_path"] = ckpt

In [75]:
model = instantiate(cfg.model)

InstantiationException: Error in call to target 'lightning.pytorch.core.module.LightningModule.load_from_checkpoint':
RuntimeError('Error(s) in loading state_dict for BasicJUMPModule:\n\tMissing key(s) in state_dict: "molecule_encoder.backbone.0.0.weight", "molecule_encoder.backbone.0.0.bias", "molecule_encoder.backbone.0.1.weight", "molecule_encoder.backbone.0.1.bias", "molecule_encoder.backbone.0.1.running_mean", "molecule_encoder.backbone.0.1.running_var", "molecule_encoder.backbone.1.0.weight", "molecule_encoder.backbone.1.0.bias", "molecule_encoder.backbone.1.1.weight", "molecule_encoder.backbone.1.1.bias", "molecule_encoder.backbone.1.1.running_mean", "molecule_encoder.backbone.1.1.running_var", "molecule_encoder.backbone.2.0.weight", "molecule_encoder.backbone.2.0.bias", "molecule_encoder.backbone.2.1.weight", "molecule_encoder.backbone.2.1.bias", "molecule_encoder.backbone.2.1.running_mean", "molecule_encoder.backbone.2.1.running_var", "molecule_encoder.backbone.3.0.weight", "m

In [ ]:
evaluators = instantiate_evaluator_list(cfg.eval, cross_modal_module=None, logger=None)

In [73]:
instantiate_evaluator_list?

Signature:
instantiate_evaluator_list(
    evaluator_list_cfg: omegaconf.dictconfig.DictConfig,
    cross_modal_module: torch.nn.modules.module.Module,
    logger: Optional[List[lightning.pytorch.loggers.logger.Logger]] = None,
    ckpt_path: Optional[str] = None,
    name: Optional[str] = None,
)
Docstring: Instantiates evaluator list from config.
File:      /mnt/2547d4d7-6732-4154-b0e1-17b0c1e0c565/Document-2/Projet2/Stage/workspace/jump_models/src/utils/instantiators.py
Type:      function

In [36]:
logger = utils.instantiate_loggers(cfg.get("logger"))

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: gabriel-watkinson-work (jump_models). Use `wandb login --relogin` to force relogin


In [83]:
Trainer?

Init signature:
Trainer(
    *,
    accelerator: Union[str, lightning.pytorch.accelerators.accelerator.Accelerator] = 'auto',
    strategy: Union[str, lightning.pytorch.strategies.strategy.Strategy] = 'auto',
    devices: Union[List[int], str, int] = 'auto',
    num_nodes: int = 1,
    precision: Union[Literal[64, 32, 16], Literal['16-mixed', 'bf16-mixed', '32-true', '64-true'], Literal['64', '32', '16', 'bf16']] = '32-true',
    logger: Union[lightning.pytorch.loggers.logger.Logger, Iterable[lightning.pytorch.loggers.logger.Logger], bool, NoneType] = None,
    callbacks: Union[List[lightning.pytorch.callbacks.callback.Callback], lightning.pytorch.callbacks.callback.Callback, NoneType] = None,
    fast_dev_run: Union[int, bool] = False,
    max_epochs: Optional[int] = None,
    min_epochs: Optional[int] = None,
    max_steps: int = -1,
    min_steps: Optional[int] = None,
    max_time: Union[str, datetime.timedelta, Dict[str, int], NoneType] = None,
    limit_train_batches: Union[int, 

In [85]:
trainer._logger_connector.log_metrics?

Signature:
trainer._logger_connector.log_metrics(
    metrics: Dict[str, torch.Tensor],
    step: Optional[int] = None,
) -> None
Docstring:
Logs the metric dict passed in. If `step` parameter is None and `step` key is presented is metrics, uses
metrics["step"] as a step.

Args:
    metrics: Metric values
    step: Step for which metrics should be logged. Default value is `self.global_step` during training or
        the total validation / test log step count during validation and testing.
File:      ~/miniconda3/envs/jump_models/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py
Type:      method

In [88]:
dl

In [81]:
trainer = instantiate(cfg.trainer, logger=logger, name="test")

InstantiationException: Error in call to target 'lightning.pytorch.trainer.trainer.Trainer':
TypeError("Trainer.__init__() got an unexpected keyword argument 'name'")
full_key: trainer

In [56]:
[x for x in trainer.loggers[2].__dir__() if "log" in x]

['_log_model',
 '_logged_model_time',
 'log_hyperparams',
 'log_metrics',
 'log_table',
 'log_text',
 'log_image',
 '_scan_and_log_checkpoints',
 'log_dir',
 'log_graph']

In [59]:
trainer.loggers[2].log_image?

Signature:
trainer.loggers[2].log_image(
    key: str,
    images: List[Any],
    step: Optional[int] = None,
    **kwargs: Any,
) -> None
Docstring:
Log images (tensors, numpy arrays, PIL Images or file paths).

Optional kwargs are lists passed to each image (ex: caption, masks, boxes).
File:      ~/miniconda3/envs/jump_models/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py
Type:      method

In [4]:
dotenv.load_dotenv()

True

In [5]:
api = wandb.Api()

run = api.run("gabriel-watkinson-work/fp_small/pgwoh2f5")

KeyError: 'accuracy'

In [ ]:
if run.state == "finished":
    for i, row in run.history().iterrows():
        print(row["_timestamp"], row["accuracy"])

In [10]:
list(run.summary.keys())

['gradients/image_encoder.backbone.layer1.0.conv1.weight',
 'gradients/image_encoder.backbone.layer2.1.conv2.weight',
 'gradients/molecule_encoder.backbone.2.1.weight',
 '_wandb',
 'gradients/image_encoder.backbone.layer1.1.bn2.bias',
 'gradients/image_encoder.backbone.layer2.0.downsample.1.weight',
 'jump_moa/image/val/Accuracy_top_3',
 'gradients/image_encoder.backbone.layer2.1.bn1.weight',
 'jump_moa/image/val/F1Score_top_5',
 'gradients/image_encoder.backbone.layer1.0.bn2.weight',
 'gradients/image_encoder.backbone.layer2.1.bn2.bias',
 'gradients/image_encoder.backbone.layer2.1.conv1.weight',
 'gradients/molecule_encoder.backbone.4.0.weight',
 'gradients/image_encoder.backbone.layer1.1.bn1.bias',
 'gradients/image_encoder.backbone.layer2.1.bn2.weight',
 'gradients/image_encoder.backbone.layer2.1.bn1.bias',
 'gradients/image_encoder.backbone.layer3.0.downsample.1.bias',
 'gradients/image_encoder.backbone.bn1.weight',
 'gradients/image_encoder.backbone.layer4.1.conv2.weight',
 'lr-Ad